# 1.0 Tensors basics

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

## Visual studio tips
1. scalar (1 number), vector (1 dimensional), matrix (2 dimensional)
2. Crtl+Shift+V to paste text from other sources
3. Shift+Enter to display markdown

## Basic functions
tf.ones(shape)
tf.zeros(shape)

## Markdown

1. number of # represent order of heading
2. $ equation $
3. *italic*
4. **bold**
5. next line \
* bullets

## Tensors

* A tensor is a generalization of vectors and matrices to potentially higher dimensions. Internally, TensorFlow represents tensors as n-dimensional arrays of base datatypes
* Each tensor represents a partialy defined computation that will eventually produce a value
* TensorFlow programs work by building a **graph** of Tensor objects that details how tensors are related. Running different parts of the **graph** allow results to be generated
* Each tensor has a data **type** and a **shape**
* `dtype`: 
  - `tf.string`: String variable (The b prefix is to indicate byte strings rather than unicode strings)    
  - `tf.float32`: Float variable    
  - `tf.int32`: Integer variable
  - `tf.bool`: boolean, true or false
* name: Optional, "Const_1:0" by default

### Types of tensors
tf.constant(value, dtype, name = "")\
tf.Variable(value, dtype, name = "")\
tf.placeholder(value, dtype, name = "")\
tf.SparseTensor(value, dtype, name = "")\
- constance can't be changed while variables can

In [ ]:
r1 = tf.constant([1,2,3], tf.int16) 
print(r1)

In [ ]:
r2 = tf.Variable([["test", "ok"], ["test", "yes"]], tf.string)
print(r2)

In [ ]:
# Creating a 2D tensor
matrix = [[1,2,3,4,5],
          [6,7,8,9,10],
          [11,12,13,14,15],
          [16,17,18,19,20]]

tensor = tf.Variable(matrix, dtype=tf.int32) 
print(tf.rank(tensor))
print(tensor.shape)

In [ ]:
print(tensor[0,2]) 
row1 = tensor[0]  # selects the first row
print(row1)
column1 = tensor[:, 0]  # selects the first column
print(column1)
row2and4 = tensor[1::2]  # selects second and fourth row
print(row2and4)
# a[start:end:step] means. a[1::2]=get every odd index, a[::2]=get every even index
# a[2::2]=get every even starting at 2, a[2:4:2]=get every even starting at 2 and ending at 4

### Degree/rank of tensors
the number of tensor dimension, 0=scalar,1=vector,2=matrix...\
tf.rank()

In [ ]:
tf.rank(r1)

In [ ]:
tf.rank(r2)

### Tensor shape
.shape\
tf.shape()\
tf.reshape()

In [ ]:
r2.shape

In [ ]:
r3 = tf.reshape(r2, [1,4])
r4 = tf.reshape(r2, [4,-1])           # -1 tells the tensor to automatically calculate the size according to the first assigntment of 4
print(r3)
print(r4)

### Evalualte tensor
examine the value of tensor from time to time\
we need to run a *session* to evalue the tensors since they represent a partially complete computation\
the default graph holds all operations not speficied to any other graph. To evaluate the tensor stored in the default graph:

In [ ]:
with tf.Session() as sess:
    r4.eval()

# 2.0 Core Learning Algorithms

- Linear Regression
- Classification
- Clustering
- Hidden Markov Models

## 2.1 Linear Regression

In [ ]:
pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = [1, 2, 2.5, 3, 4]
y = [1, 4, 7, 9, 15]
plt.plot(x, y, 'ro')

### Direct call of np.polyfit()
$ \hat y = a+b*X$

In [ ]:
plt.plot(x, y, 'ro')
plt.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)))   # np.unique() removes duplicate elements

### Train the model to do linear regression

#### Load a dataset
required libraries: 
- numpy for multidimensional array calculation
- pandas for dataset manipulating
  - df.shape
  - df.head(): show the first 5 items
  - df.describe(): return statistics
  - df.lable.hist() or df['lable'].hist()
  - df.lable.value_counts() 
  - df.loc[*row_number*]
- sklearn for 1)Preprocessing 2)Regression 3)Classification 4)Clustering 5)Model Selection 6)Dimensionality Reduction

In [ ]:
!pip install -q sklearn

In [ ]:
pip install pandas

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [ ]:
# Load dataset.
df_train = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
df_test = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
y_train = df_train.pop('survived')
y_test = df_test.pop('survived')
# pop() method removes the element at the specified position and stored it in y.
# survive or not is the output of this titanic model

In [ ]:
df_train.head()

In [ ]:
df_train.loc[0]

In [ ]:
df_train.age.hist(bins=20)

In [ ]:
df_train.sex.value_counts().plot(kind='barh')

#### Pre-process dataset
Dataset comprises of categorical (text) and numerical (number) data. To train the model, categorical data can be converted to numbers via TensorFlow tools:
1. convert panda dataframe to feature colunmns:
    tf.feature_column.categorical_column_with_vocabulary_list()\
    tf.feature_column.numeric_column()
2. convert feature columns to a ```tf.data.Dataset``` object

In [ ]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []

for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = df_train[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)  

In [ ]:
# input function defines how much data is split to batches and epochs to feed the training model
# if data is too big might cause slow, difficult training and overfitting

def make_input_fn(data_df, output_df, num_epochs=10, shuffle=True, batch_size=32): # num_epochs, shuffle, batch_size are default and can be changed when being called
  def input_function():                                                            # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), output_df))            # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)                                                        # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)                                   # split dataset into batches of 32 and repeat process for number of epochs
    return ds                                                                      # return a batch of the dataset
  return input_function                                                            # return a function object for use

# here we will call the input_function that was returned to us to get a dataset object we can feed to the model
train_input_fn = make_input_fn(df_train, y_train)                                  # shuffle >> reorganize the order of the items  
test_input_fn = make_input_fn(df_test, y_test, num_epochs=1, shuffle=False)        # we don't shuffle y as we don't train them

#### Training the model
`tf.estimator.LinearClassifier()`\
`model.train()`\
`model.evaluate()`\
`model.fit()`

In [ ]:
# Create the model and pass the feature columns we created earlier
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

# clears consoke output
clear_output()

In [ ]:
# Train the model
linear_est.train(train_input_fn) 

# Returns the loss value & metrics values for the model in test mode
result = linear_est.evaluate(test_input_fn) 

clear_output() 

# statistics about our model. accuracy varies within [0,1]
print(result)

#### Prediction based on the trained model
`model.predict()`

In [ ]:
# Generates output predictions for the input samples & convert it to a list
prediction = list(linear_est.predict(test_input_fn))
clear_output() 
print(prediction[0])

In [ ]:
# 'probabilities' indicate the survival chance  of the 4th passenger
print(prediction[3]['probabilities'][0])
# compare it with actual survival
print(y_test.loc[3])

# 3.0 Neural Networks

- NN takes inputs and maps they to outputs
- ``Keras`` is introduced as a high-level neural networks API
- $Y =activation[((\sum_{i=0}^n w_i) x_i) + b]$
- for every neuron in the next layer, there's only one bias from the previous layer
- actiavation function returns a value between [0,1] for any vlalue of weighted sums + bias from the previous layer. 
  
- Common activation functions: 
  - `Relu` (Rectified Linear Unit)
  - `Tanh` (Hyperbolic Tangent)
  - `Sigmoid`
  - ...
- Common types of data:
  - Vector Data (2D)
  - Timeseries or Sequence (3D)
  - Image Data (4D)
  - Video Data (5D)

- **loss/cost functions**: For our training data we have the features (input) and the labels (expected output). The difference between the output from our network to the expected output is called loss functions. Common loss/cost functions include.
  - Mean Squared Error (MSE, default)
  - Mean Absolute Error (MAE)
  - Hinge Loss (HL)

- **Optimization function**: algorithm used to update weights and biases while backpropagation, to find the optimal paramaters (weights and biases) for our network. Common optimizers:
  - Gradient Descent (GD)
  - Stochastic Gradient Descent (SGD)
  - Mini-Batch Gradient Descent
  - Momentum
  - Adam (momentum and GD combined)
(https://medium.com/@sdoshi579/optimizers-for-training-neural-network-59450d71caf6)

GD determines which direction to go to get the global minimal, and the backpropagation process goes to that direction and updates weights and biases

## Create a NN

In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

a MNIST Fashion example: total 70000 images clothes are supposed to be classified into 10 categories.

In [10]:
# load dataset
fashion_mnist = keras.datasets.fashion_mnist  

# split into tetsing and training
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()  

print(train_images.shape)  # 60000 images of 28*28 pixels, varing in [0,255]
print(test_images.shape)   # 10000 images of 28*28 pixels, varing in [0,255]
print(train_labels.shape)  # 60000 values for clothe caterogies, varing in [0,9] 
print(test_labels.shape)   # 10000 values for clothe caterogies, varing in [0,9] 

(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)


In [ ]:
# to pull out a random training image
# plt.figure()
# plt.imshow(train_images[1])
# plt.colorbar()
# plt.show()
####### I don't know why but this is problematic. Don't run it

In [11]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [12]:
print(type(train_images))

# data normalization
train_images = train_images / 255.0
test_images = test_images / 255.0

<class 'numpy.ndarray'>


## Create a NN

model = `keras.Sequential()`:\
model.add()\
model.complie()\
model.evaluate()\
model.fit(x,y,batch_size, epochs): Trains the model for a fixed number of epochs\
model.predict(x)

In [16]:
# Build a 3-layer sequtial model

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),  # input layer 
    keras.layers.Dense(128, activation='relu'),  # hidden layer 
    keras.layers.Dense(10, activation='softmax') # output layer 
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10)  

Epoch 1/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.4950 - accuracy: 0.8273
Epoch 2/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.3681 - accuracy: 0.8672
Epoch 3/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.3357 - accuracy: 0.8781
Epoch 4/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.3120 - accuracy: 0.8838
Epoch 5/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2925 - accuracy: 0.8916
Epoch 6/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2803 - accuracy: 0.8958
Epoch 7/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.2686 - accuracy: 0.8995
Epoch 8/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.2578 - accuracy: 0.9041
Epoch 9/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2489 - accuracy: 0.9079
Epoch 10/10
1875/1875 [==============================] - 14s 8ms/step 

- flatten layer: 1D
- Dense layers: all neurons in the previous layer connect to the every neurons in the next layer
- `'softmax'`: common activation functinos for output layer to make sure all outputs of neurons add up to 1
- An epoch is an iteration over the entire x and y data (forward and backward passes)
- Hyperparameters are not trained in this case (but they can be trained if necessary):numbers of neuron, epoch, layer, and activation functions. Only weights and biases are trained during this 10 iterations


`but why the total number of sample is 1875 instead of 6000?`

In [17]:
# The final accuracy of trained data is 0.9107
# Now we want to evalueated the trained model on testing data
# which is lower than the accuaracy of trained data >> overfitting

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=1) 
print('Test accuracy:', test_acc)

313/313 [==============================] - 5s 15ms/step - loss: 0.3489 - accuracy: 0.8738
Test accuracy: 0.8737999796867371


*the fianl goals is to increase test accuracy instead of traning accuracy*

In [21]:
# Making predictions based on the test data
predictions = model.predict(test_images)

313/313 [==============================] - 3s 11ms/step


In [26]:
print(predictions[0])
print(class_names[np.argmax(predictions[0])])
# np.argmax() returns the indices of the maximum values along an axis.
print(class_names[test_labels[0]])

[1.41598036e-07 1.79529974e-10 2.69420070e-10 1.87976842e-10
 6.59264572e-07 8.01480317e-04 1.24198793e-08 6.21224474e-03
 1.15325065e-07 9.92985249e-01]
Ankle boot
Ankle boot


## Verifying Predictions
I've written a small function here to help us verify predictions with some simple visuals.

In [ ]:
COLOR = 'white'
plt.rcParams['text.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR

def predict(model, image, correct_label):
  class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
  prediction = model.predict(np.array([image]))
  predicted_class = class_names[np.argmax(prediction)]

  show_image(image, class_names[correct_label], predicted_class)


def show_image(img, label, guess):
  plt.figure()
  plt.imshow(img, cmap=plt.cm.binary)
  plt.title("Excpected: " + label)
  plt.xlabel("Guess: " + guess)
  plt.colorbar()
  plt.grid(False)
  plt.show()


def get_number():
  while True:
    num = input("Pick a number: ")
    if num.isdigit():
      num = int(num)
      if 0 <= num <= 1000:
        return int(num)
    else:
      print("Try again...")

num = get_number()
image = test_images[num]
label = test_labels[num]
predict(model, image, label)


# 4.0 PANDAS basics

# SA-PINN frame

In [ ]:
import numpy as np
import scipy
from scipy.integrate import solve_ivp
from scipy.sparse import diags
import matplotlib.pyplot as plt
import tensorflow as tf

# Parameters
par = {
    'k0rg': 0.7,
    'k0rw': 1.,
    'ng': 2.0,
    'nw': 2.0,
    'sgr': 0.,
    'swr': 0.2,
    'μg': 0.02,
    'μw': 1.0,
    'sgi': 0.,
}

def Krw(S, swr, sgr, k0rw, nw):
    return k0rw * ((S - swr) / (1 - sgr - swr)) ** nw

def Krw_p(S, p):
    return Krw(S, p['swr'], p['sgr'], p['k0rw'], p['nw'])

def Krg(S, swr, sgr, k0rg, ng):
    return k0rg * ((1 - S - sgr) / (1 - sgr - swr)) ** ng

def Krg_p(S, p):
    return Krg(S, p['swr'], p['sgr'], p['k0rg'], p['ng'])

def f(S, p):
    return 1. / (1. + Krw_p(S, p) * p['μg'] / (Krg_p(S, p) * p['μw']))

# Solve PDE
dx = 0.01
x_points = np.arange(0, 1.01, dx)
t_points = np.linspace(0, 0.6, 61)

# Discretize the PDE
A = diags([1, -2, 1], [-1, 0, 1], shape=(len(x_points), len(x_points))).toarray()
A[0, 0] = -1
A[-1, -1] = -1
A = A / dx**2

def pde(t, S):
    dSdt = A @ S
    dSdt[0] = 1.0 - par['swr']
    return dSdt

# Initial condition
S0 = np.full(len(x_points), par['sgi'])

sol = solve_ivp(pde, (0, 0.6), S0, t_eval=t_points)

# Plotting
x_ticks = np.arange(0, 1.01, 0.01)
ts = [0.12, 0.23, 0.35, 0.47, 0.59]

fig, ax = plt.subplots()
for t_ in ts:
    t_idx = np.argmin(np.abs(sol.t - t_))
    ax.plot(x_ticks, sol.y[:, t_idx], label=f't={t_}')
ax.set_xlabel('x')
ax.set_ylabel('S(x,t)')
ax.legend()
plt.show()


In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

# or
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Dropout
from tensorflow.keras import layers, activations

model = Sequential()
model.add(layers.Flatten(input_shape=(28, 28)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10))